In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
df1= pd.read_csv("Desktop/emoint tweet/sadness-ratings-0to1.train.txt", delimiter='\t', header=None)
df1.columns = ['Id', 'tweet', 'emotion', 'score']

In [3]:
print(df1)


        Id                                              tweet  emotion  score
0    40000                      Depression sucks! #depression  sadness  0.958
1    40001            Feeling worthless as always #depression  sadness  0.958
2    40002                       Feeling worthless as always   sadness  0.958
3    40003  My #Fibromyalgia has been really bad lately wh...  sadness  0.946
4    40004  Im think ima lay in bed all day and sulk. Life...  sadness  0.934
..     ...                                                ...      ...    ...
781  40781  @VivienLloyd Thank you so much! Just home - st...  sadness  0.104
782  40782              Just put the winter duvet on ☃️❄️🌬☔️   sadness  0.104
783  40783  @SilkInSide @TommyJoeRatliff that's so pretty!...  sadness  0.088
784  40784  @BluesfestByron second artist announcement loo...  sadness  0.083
785  40785  I can literally eat creamy pesto pasta topped ...  sadness  0.083

[786 rows x 4 columns]


In [4]:
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import emoji
import re

In [5]:
def preprocess_tweet(tweet):
    tweet = tweet.lower()
    tweet = re.sub(r"http\S+|www\S+|https\S+", "", tweet)
    
    return tweet
df1['tweet'] = df1['tweet'].apply(preprocess_tweet)

In [6]:
def remove_mentions(tweet):
    return re.sub(r'@\w+', '', tweet)

df1['tweet'] = df1['tweet'].apply(remove_mentions)


In [7]:
def clean_text(text):
    text = re.sub(r'[^\w\s#@]', '', text) 
    text = re.sub(r'\d+', '', text) 
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df1['tweet'] = df1['tweet'].apply(clean_text)



In [8]:
df1

,Id,tweet,emotion,score
0,40000,depression sucks #depression,sadness,0.958
1,40001,feeling worthless as always #depression,sadness,0.958
2,40002,feeling worthless as always,sadness,0.958
3,40003,my #fibromyalgia has been really bad lately wh...,sadness,0.946
4,40004,im think ima lay in bed all day and sulk life ...,sadness,0.934
...,...,...,...,...
781,40781,thank you so much just home stunned but so hap...,sadness,0.104
782,40782,just put the winter duvet on,sadness,0.104
783,40783,thats so pretty i love the sky in the backgrou...,sadness,0.088
784,40784,second artist announcement looking good #blues...,sadness,0.083


In [9]:
def convert_emoji(tweet):
    text = emoji.demojize(tweet)
    return text

In [10]:
df1['tweet'] = df1['tweet'].apply(convert_emoji)

In [11]:
def tokenize_tweets(tweet):
    tokenizer = TweetTokenizer()
    return tokenizer.tokenize(tweet)

df1['tweet'] = df1['tweet'].apply(tokenize_tweets)

In [12]:
df1

,Id,tweet,emotion,score
0,40000,"[depression, sucks, #depression]",sadness,0.958
1,40001,"[feeling, worthless, as, always, #depression]",sadness,0.958
2,40002,"[feeling, worthless, as, always]",sadness,0.958
3,40003,"[my, #fibromyalgia, has, been, really, bad, la...",sadness,0.946
4,40004,"[im, think, ima, lay, in, bed, all, day, and, ...",sadness,0.934
...,...,...,...,...
781,40781,"[thank, you, so, much, just, home, stunned, bu...",sadness,0.104
782,40782,"[just, put, the, winter, duvet, on]",sadness,0.104
783,40783,"[thats, so, pretty, i, love, the, sky, in, the...",sadness,0.088
784,40784,"[second, artist, announcement, looking, good, ...",sadness,0.083


In [13]:
stop_words = [
    'a', 'an', 'and', 'are', 'as', 'at', 'be', 'by', 'for', 'from', 'has', 'he', 'in', 'is', 'it', 'its',
    'of', 'on', 'that', 'the', 'to', 'was', 'were', 'will', 'with', 'i', 'you', 'your', 'so', 'this', 'all',
    'am', 'or', 'but', 'if', 'my', 'me', 'we', 'us', 'our', 'we', 'up', 'down', 'out', 'just', 'how', 'why',
    'when', 'where', 'here', 'there', 'about', 'more', 'most', 'some', 'any', 'few', 'many', 'much', 'not',
    'only', 'other', 'same', 'such', 'no', 'nor', 'too', 'very', 'can', 'cannot', 'could', 'should', 'would',
    'might', 'must', 'shall', 'will', 'isn', 'hasn', 'doesn', 'haven', 'didn', 'hadn', 'wasn', 'weren',
    'wouldn', 'shouldn', 'ain', 'aren', 'ma'
]

def remove_stopwords(tokens):
    filtered_tokens = [token for token in tokens if len(token) > 1 and token.lower() not in stop_words]
    return filtered_tokens

df1['tweet'] = df1['tweet'].apply(remove_stopwords)



In [14]:
df1


,Id,tweet,emotion,score
0,40000,"[depression, sucks, #depression]",sadness,0.958
1,40001,"[feeling, worthless, always, #depression]",sadness,0.958
2,40002,"[feeling, worthless, always]",sadness,0.958
3,40003,"[#fibromyalgia, been, really, bad, lately, whi...",sadness,0.946
4,40004,"[im, think, ima, lay, bed, day, sulk, life, hi...",sadness,0.934
...,...,...,...,...
781,40781,"[thank, home, stunned, happy, dont, think, sun...",sadness,0.104
782,40782,"[put, winter, duvet]",sadness,0.104
783,40783,"[thats, pretty, love, sky, background, purple,...",sadness,0.088
784,40784,"[second, artist, announcement, looking, good, ...",sadness,0.083


In [15]:
import torch
from transformers import BertModel, BertTokenizer



model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)


def extract_features(tweet):
    
    tokenized_text = ' '.join(tweet)

    input_ids = torch.tensor(tokenizer.encode(tokenized_text, add_special_tokens=True)).unsqueeze(0)
    
    
    with torch.no_grad():
        outputs = model(input_ids)
    
   
    last_hidden_states = outputs.last_hidden_state
    
    features = last_hidden_states.squeeze(0).numpy()
    
    return features


df1['features'] = df1['tweet'].apply(extract_features)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [16]:
df1

,Id,tweet,emotion,score,features
0,40000,"[depression, sucks, #depression]",sadness,0.958,"[[-0.1279724, 0.41236815, 0.051530708, -0.1606..."
1,40001,"[feeling, worthless, always, #depression]",sadness,0.958,"[[-0.12421638, 0.25974998, -0.0661158, -0.3355..."
2,40002,"[feeling, worthless, always]",sadness,0.958,"[[-0.23523621, 0.27867872, -0.5598585, -0.2093..."
3,40003,"[#fibromyalgia, been, really, bad, lately, whi...",sadness,0.946,"[[-0.04965788, 0.24742632, -0.007569845, -0.58..."
4,40004,"[im, think, ima, lay, bed, day, sulk, life, hi...",sadness,0.934,"[[-0.04689168, 0.025870707, 0.33319533, -0.149..."
...,...,...,...,...,...
781,40781,"[thank, home, stunned, happy, dont, think, sun...",sadness,0.104,"[[-0.042937506, 0.24438974, 0.10452047, -0.057..."
782,40782,"[put, winter, duvet]",sadness,0.104,"[[-0.18927328, 0.033073332, -0.024956549, 0.17..."
783,40783,"[thats, pretty, love, sky, background, purple,...",sadness,0.088,"[[-0.39431044, -0.13764822, -0.2091385, -0.067..."
784,40784,"[second, artist, announcement, looking, good, ...",sadness,0.083,"[[-0.21641317, 0.026052019, -0.18617798, -0.03..."


In [17]:
df1.features[0].shape

(6, 768)

In [18]:

df1.head(10)

,Id,tweet,emotion,score,features
0,40000,"[depression, sucks, #depression]",sadness,0.958,"[[-0.1279724, 0.41236815, 0.051530708, -0.1606..."
1,40001,"[feeling, worthless, always, #depression]",sadness,0.958,"[[-0.12421638, 0.25974998, -0.0661158, -0.3355..."
2,40002,"[feeling, worthless, always]",sadness,0.958,"[[-0.23523621, 0.27867872, -0.5598585, -0.2093..."
3,40003,"[#fibromyalgia, been, really, bad, lately, whi...",sadness,0.946,"[[-0.04965788, 0.24742632, -0.007569845, -0.58..."
4,40004,"[im, think, ima, lay, bed, day, sulk, life, hi...",sadness,0.934,"[[-0.04689168, 0.025870707, 0.33319533, -0.149..."
5,40005,"[try, fail, dont, try, still, fail, #confused,...",sadness,0.917,"[[-0.30267218, 0.38377964, -0.13703564, 0.1354..."
6,40006,"[extreme, sadness]",sadness,0.917,"[[-0.166719, 0.14257, -0.1267908, 0.038275078,..."
7,40007,"[life, one, word, depressing]",sadness,0.917,"[[-0.030349113, 0.13385968, -0.20066914, 0.058..."
8,40008,"[panic, attacks, worst, feeling, really, sick,...",sadness,0.917,"[[-0.31141603, 0.07427356, 0.34359968, -0.3796..."
9,40009,"[feel, grim, ugly, atm]",sadness,0.896,"[[-0.04971684, 0.49060404, 0.027603516, -0.081..."


In [19]:
from tensorflow.keras.preprocessing.sequence import pad_sequences


features = df1['features'].tolist()


padded_features = pad_sequences(features, padding='post')


padded_df = df1.copy()
padded_df['features'] = padded_features.tolist()

In [20]:
X = np.stack(padded_df['features'])

print('Input feature shape:', X.shape)

Input feature shape: (786, 41, 768)


In [21]:
y = np.array(padded_df['score'])    
print("Input shape:", X.shape)
print("Output shape:", y.shape)

Input shape: (786, 41, 768)
Output shape: (786,)


In [23]:
X = np.reshape(X, (786, 41* 768))  
y = np.reshape(y, (786,))

In [24]:
X.shape

(786, 31488)

In [25]:
Y=df1['score'].copy()

     

In [26]:
import tensorflow as tf

In [29]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPooling1D
from keras.callbacks import EarlyStopping

model = Sequential()
model.add(Dense(512, input_shape=(31488,), activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1, activation='linear'))



early_stopping = EarlyStopping(monitor='mean_squared_error', patience=1, mode='min')
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 512)               16122368  
                                                                 
 dropout_5 (Dropout)         (None, 512)               0         
                                                                 
 dense_7 (Dense)             (None, 256)               131328    
                                                                 
 dropout_6 (Dropout)         (None, 256)               0         
                                                                 
 dense_8 (Dense)             (None, 128)               32896     
                                                                 
 dropout_7 (Dropout)         (None, 128)               0         
                                                                 
 dense_9 (Dense)             (None, 64)               

In [30]:

df1['tweet'] = df1['tweet'].apply(lambda tokens: ' '.join(tokens))

array = df1['tweet'].values

tensor = tf.convert_to_tensor(array)




In [31]:

# Train the model with the EarlyStopping callback
history = model.fit(X, Y, batch_size=25, epochs=15, shuffle=True, verbose=1, callbacks=[early_stopping])


Epoch 1/15
32/32 [==============================] - 7s 184ms/step - loss: 0.0741 - mean_squared_error: 0.0741
Epoch 2/15
32/32 [==============================] - 6s 181ms/step - loss: 0.0576 - mean_squared_error: 0.0576
Epoch 3/15
32/32 [==============================] - 6s 181ms/step - loss: 0.0376 - mean_squared_error: 0.0376
Epoch 4/15
32/32 [==============================] - 6s 182ms/step - loss: 0.0301 - mean_squared_error: 0.0301
Epoch 5/15
32/32 [==============================] - 6s 182ms/step - loss: 0.0257 - mean_squared_error: 0.0257
Epoch 6/15
32/32 [==============================] - 6s 179ms/step - loss: 0.0228 - mean_squared_error: 0.0228
Epoch 7/15
32/32 [==============================] - 6s 183ms/step - loss: 0.0236 - mean_squared_error: 0.0236


In [32]:
model.save('emointjoy.h5','/Home')

In [33]:
from keras.models import load_model

loaded_model = load_model('emointjoy.h5')

In [34]:
test= pd.read_csv("Desktop/emoint tweet/sadness-ratings-0to1.test.gold.txt", delimiter='\t', header=None)
test.columns = ['Id', 'tweet', 'emotion', 'score']

In [35]:
test

,Id,tweet,emotion,score
0,40860,My 2 teens sons just left in the car to get ha...,sadness,0.667
1,40861,My 2 teens sons just left in the car to get ha...,sadness,0.458
2,40862,HartRamsey'sUPLIFT If you're still discouraged...,sadness,0.396
3,40863,@AmontanaW I nearly dropped my phone into the ...,sadness,0.271
4,40864,Whenever I'm feeling sad I will listen to mons...,sadness,0.604
...,...,...,...,...
668,41528,Why does Candice constantly pout #GBBO 💄😒,sadness,0.396
669,41529,"@redBus_in #unhappy with #redbus CC, when I ta...",sadness,0.604
670,41530,"@AceOperative789 no pull him afew weeks ago, s...",sadness,0.479
671,41531,I'm buying art supplies and I'm debating how s...,sadness,0.375


In [36]:
test= test.drop("score",axis=1)

In [37]:
test

,Id,tweet,emotion
0,40860,My 2 teens sons just left in the car to get ha...,sadness
1,40861,My 2 teens sons just left in the car to get ha...,sadness
2,40862,HartRamsey'sUPLIFT If you're still discouraged...,sadness
3,40863,@AmontanaW I nearly dropped my phone into the ...,sadness
4,40864,Whenever I'm feeling sad I will listen to mons...,sadness
...,...,...,...
668,41528,Why does Candice constantly pout #GBBO 💄😒,sadness
669,41529,"@redBus_in #unhappy with #redbus CC, when I ta...",sadness
670,41530,"@AceOperative789 no pull him afew weeks ago, s...",sadness
671,41531,I'm buying art supplies and I'm debating how s...,sadness


In [38]:
def preprocess_tweet(tweet):
    tweet = tweet.lower()
    tweet = re.sub(r"http\S+|www\S+|https\S+", "", tweet)
    
    return tweet
test['tweet'] = test['tweet'].apply(preprocess_tweet)

In [39]:
test

,Id,tweet,emotion
0,40860,my 2 teens sons just left in the car to get ha...,sadness
1,40861,my 2 teens sons just left in the car to get ha...,sadness
2,40862,hartramsey'suplift if you're still discouraged...,sadness
3,40863,@amontanaw i nearly dropped my phone into the ...,sadness
4,40864,whenever i'm feeling sad i will listen to mons...,sadness
...,...,...,...
668,41528,why does candice constantly pout #gbbo 💄😒,sadness
669,41529,"@redbus_in #unhappy with #redbus cc, when i ta...",sadness
670,41530,"@aceoperative789 no pull him afew weeks ago, s...",sadness
671,41531,i'm buying art supplies and i'm debating how s...,sadness


In [40]:
def remove_mentions(tweet):
    return re.sub(r'@\w+', '', tweet)

test['tweet'] = test['tweet'].apply(remove_mentions)
test

,Id,tweet,emotion
0,40860,my 2 teens sons just left in the car to get ha...,sadness
1,40861,my 2 teens sons just left in the car to get ha...,sadness
2,40862,hartramsey'suplift if you're still discouraged...,sadness
3,40863,i nearly dropped my phone into the sink hahahaha,sadness
4,40864,whenever i'm feeling sad i will listen to mons...,sadness
...,...,...,...
668,41528,why does candice constantly pout #gbbo 💄😒,sadness
669,41529,"#unhappy with #redbus cc, when i talked with ...",sadness
670,41530,"no pull him afew weeks ago, sadly theres no g...",sadness
671,41531,i'm buying art supplies and i'm debating how s...,sadness


In [41]:
def clean_text(text):
    text = re.sub(r'[^\w\s#@]', '', text)  # Remove all non-word characters except # and @ symbols
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = re.sub(r'\s+', ' ', text).strip()
    return text

test['tweet'] = test['tweet'].apply(clean_text)


In [42]:
test

,Id,tweet,emotion
0,40860,my teens sons just left in the car to get hair...,sadness
1,40861,my teens sons just left in the car to get hair...,sadness
2,40862,hartramseysuplift if youre still discouraged i...,sadness
3,40863,i nearly dropped my phone into the sink hahahaha,sadness
4,40864,whenever im feeling sad i will listen to monst...,sadness
...,...,...,...
668,41528,why does candice constantly pout #gbbo,sadness
669,41529,#unhappy with #redbus cc when i talked with th...,sadness
670,41530,no pull him afew weeks ago sadly theres no gam...,sadness
671,41531,im buying art supplies and im debating how ser...,sadness


In [43]:
def convert_emoji(tweet):
    text = emoji.demojize(tweet)
    return text

test['tweet'] = test['tweet'].apply(convert_emoji)

In [44]:
def tokenize_tweets(tweet):
    tokenizer = TweetTokenizer()
    return tokenizer.tokenize(tweet)

test['tweet'] = test['tweet'].apply(tokenize_tweets)

In [45]:
test

,Id,tweet,emotion
0,40860,"[my, teens, sons, just, left, in, the, car, to...",sadness
1,40861,"[my, teens, sons, just, left, in, the, car, to...",sadness
2,40862,"[hartramseysuplift, if, youre, still, discoura...",sadness
3,40863,"[i, nearly, dropped, my, phone, into, the, sin...",sadness
4,40864,"[whenever, im, feeling, sad, i, will, listen, ...",sadness
...,...,...,...
668,41528,"[why, does, candice, constantly, pout, #gbbo]",sadness
669,41529,"[#unhappy, with, #redbus, cc, when, i, talked,...",sadness
670,41530,"[no, pull, him, afew, weeks, ago, sadly, there...",sadness
671,41531,"[im, buying, art, supplies, and, im, debating,...",sadness


In [46]:

stop_words = [
    'a', 'an', 'and', 'are', 'as', 'at', 'be', 'by', 'for', 'from', 'has', 'he', 'in', 'is', 'it', 'its',
    'of', 'on', 'that', 'the', 'to', 'was', 'were', 'will', 'with', 'i', 'you', 'your', 'so', 'this', 'all',
    'am', 'or', 'but', 'if', 'my', 'me', 'we', 'us', 'our', 'we', 'up', 'down', 'out', 'just', 'how', 'why',
    'when', 'where', 'here', 'there', 'about', 'more', 'most', 'some', 'any', 'few', 'many', 'much', 'not',
    'only', 'other', 'same', 'such', 'no', 'nor', 'too', 'very', 'can', 'cannot', 'could', 'should', 'would',
    'might', 'must', 'shall', 'will', 'isn', 'hasn', 'doesn', 'haven', 'didn', 'hadn', 'wasn', 'weren',
    'wouldn', 'shouldn', 'ain', 'aren', 'ma'
]

def remove_stopwords(tokens):
    filtered_tokens = [token for token in tokens if len(token) > 1 and token.lower() not in stop_words]
    return filtered_tokens

test['tweet'] = test['tweet'].apply(remove_stopwords)



In [47]:
test

,Id,tweet,emotion
0,40860,"[teens, sons, left, car, get, haircuts, im, pr...",sadness
1,40861,"[teens, sons, left, car, get, haircuts, im, pr...",sadness
2,40862,"[hartramseysuplift, youre, still, discouraged,...",sadness
3,40863,"[nearly, dropped, phone, into, sink, hahahaha]",sadness
4,40864,"[whenever, im, feeling, sad, listen, monsta, h...",sadness
...,...,...,...
668,41528,"[does, candice, constantly, pout, #gbbo]",sadness
669,41529,"[#unhappy, #redbus, cc, talked, them, before, ...",sadness
670,41530,"[pull, him, afew, weeks, ago, sadly, theres, g...",sadness
671,41531,"[im, buying, art, supplies, im, debating, seri...",sadness


In [48]:
import torch
from transformers import BertModel, BertTokenizer


model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

def extract_features(tweet):
    
    tokenized_text = ' '.join(tweet)
    
    
    input_ids = torch.tensor(tokenizer.encode(tokenized_text, add_special_tokens=True)).unsqueeze(0)
    
    with torch.no_grad():
        outputs = model(input_ids)
    
 
    last_hidden_states = outputs.last_hidden_state
    
    features = last_hidden_states.squeeze(0).numpy()
    
    return features


test['features'] = test['tweet'].apply(extract_features)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [49]:
test["features"][0].shape

(24, 768)

In [50]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

features = test['features'].tolist()

padded_features = pad_sequences(features, padding='post')


padded_df = test.copy()
padded_df['features'] = padded_features.tolist()

In [51]:
X_test= np.stack(padded_df['features'])


print('Input feature shape:', X_test.shape)

Input feature shape: (673, 37, 768)


In [57]:

from tensorflow.keras.preprocessing.sequence import pad_sequences

max_sequence_length = 41

padded_test_data = pad_sequences(X_test, maxlen=max_sequence_length, padding='post', truncating='post')


In [60]:
X = np.reshape(padded_test_data, (673, 41 * 768))  

In [61]:
test_features = np.array(X)

In [62]:
print(test_features.shape)

(673, 31488)


In [63]:
predictions = loaded_model.predict(test_features)

22/22 [==============================] - 1s 21ms/step


In [64]:
print(predictions)

[[0.3411812 ]
 [0.40345177]
 [0.27133304]
 [0.3518517 ]
 [0.46513572]
 [0.36193806]
 [0.3766475 ]
 [0.328167  ]
 [0.34471503]
 [0.3683497 ]
 [0.4882487 ]
 [0.3384192 ]
 [0.4150851 ]
 [0.2682984 ]
 [0.32977828]
 [0.27380416]
 [0.36888805]
 [0.50044775]
 [0.29151914]
 [0.35253394]
 [0.4180524 ]
 [0.36293766]
 [0.42845392]
 [0.4439992 ]
 [0.3007766 ]
 [0.32886606]
 [0.37609687]
 [0.3341128 ]
 [0.38928786]
 [0.42633462]
 [0.492797  ]
 [0.33978438]
 [0.44398192]
 [0.47217068]
 [0.4167275 ]
 [0.32791248]
 [0.31761992]
 [0.3815317 ]
 [0.30683723]
 [0.2382664 ]
 [0.29098478]
 [0.32508627]
 [0.4102328 ]
 [0.3829669 ]
 [0.29903734]
 [0.42061922]
 [0.4347054 ]
 [0.4395783 ]
 [0.3709918 ]
 [0.40905824]
 [0.3801019 ]
 [0.3464416 ]
 [0.43534353]
 [0.40883777]
 [0.34109363]
 [0.42957482]
 [0.24879926]
 [0.3800583 ]
 [0.35032275]
 [0.32783258]
 [0.34778216]
 [0.28116828]
 [0.48585543]
 [0.35491955]
 [0.37281895]
 [0.3751361 ]
 [0.659537  ]
 [0.4277515 ]
 [0.42955282]
 [0.3652611 ]
 [0.22983034]
 [0.39

In [65]:
test['score']=pd.DataFrame(predictions)

In [66]:
test=test.drop('features',axis=1)


In [68]:
test

,Id,tweet,emotion,score
0,40860,"[teens, sons, left, car, get, haircuts, im, pr...",sadness,0.341181
1,40861,"[teens, sons, left, car, get, haircuts, im, pr...",sadness,0.403452
2,40862,"[hartramseysuplift, youre, still, discouraged,...",sadness,0.271333
3,40863,"[nearly, dropped, phone, into, sink, hahahaha]",sadness,0.351852
4,40864,"[whenever, im, feeling, sad, listen, monsta, h...",sadness,0.465136
...,...,...,...,...
668,41528,"[does, candice, constantly, pout, #gbbo]",sadness,0.395225
669,41529,"[#unhappy, #redbus, cc, talked, them, before, ...",sadness,0.251602
670,41530,"[pull, him, afew, weeks, ago, sadly, theres, g...",sadness,0.306839
671,41531,"[im, buying, art, supplies, im, debating, seri...",sadness,0.370166
